# I. $LU$ factorization of a square matrix
When we premultply $A$ by lower triangular elemetary matrices $\Lambda_j$ it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  = U$$

$$  A  = (\Lambda_{n-1}  \ldots\Lambda_2  \Lambda_2 )^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses
$$  A  = (\Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1}) U$$

and inverse of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_j^{-1} = - \Lambda_j$, so 

$$ A  =  (-\Lambda_1) (-\Lambda_2) \ldots   (-\Lambda_{n-1})  U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1}  \ldots\Lambda_2  \Lambda_1 A  $$
an upper triangular matrix
$$ L  =  \Lambda_1^{-1}  \Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} $$
an lower triangular matrix.

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [37]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [38]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [39]:
a

array([[3.   , 3.   , 3.   , 3.   , 3.   , 3.   ],
       [3.   , 1.875, 1.364, 1.071, 0.882, 0.75 ],
       [3.   , 1.364, 0.882, 0.652, 0.517, 0.429],
       [3.   , 1.071, 0.652, 0.469, 0.366, 0.3  ],
       [3.   , 0.882, 0.517, 0.366, 0.283, 0.231],
       [3.   , 0.75 , 0.429, 0.3  , 0.231, 0.188]])

In [40]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [41]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [42]:
a1 = a.copy()
a1[1, 1] = 3

In [43]:
np.linalg.matrix_rank(a1)

6

In [44]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [45]:
def leading_minors_test(A):
  N = a.shape[0]
  for i in range(N):
    if(np.linalg.det(A[0:i,0:i]) == 0):
      return False
  return True
leading_minors_test(a), leading_minors_test(a1)

(True, False)

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

# 2. $LU$ factorization column pivoting
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and the premultply by lower triangular elemetary matrices $\Lambda_j$  it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  = U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$

with

\begin{array}{ll}L  &=  (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1}\\
&= P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\\
 &= P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1})
\end{array}



Because the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$, so


If we use 
$$  \hat{L}  = (\Lambda_{n-1}  \ldots \Lambda_2\Lambda_1)^{-1}$$
The inverse of a product of matrices is the revesre product of inverses 
$$ \hat{L} = \Lambda_1^{-1}\Lambda_2^{-1} \ldots  \Lambda_{n-1}^{-1} $$
the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$
$$ \hat{L} = (-\Lambda_1)  (-\Lambda_2) \ldots  (-\Lambda_{n-1}) $$

Then
$$    A_\pi  = \hat{L} U$$
with $A_\pi$ equal to $A$ with rows permutated acoording to some permutation $\pi$

Equiavlently 
$$   A  = (\hat{L}U)_{\pi'}$$
 $(\hat{L}U)_{\pi'}$ equal to $\hat{L}U$ with rows permutated acoording to some permutation $\pi'$

In [46]:
def diy_lu_column_pivot(a):
  N = a.shape[0]

  u =a.copy()
  l =np.eye(N)
  p = np.eye(N)

  for j in range(N-1):
    row = np.argmax(np.abs(u[j:,j]))+j
    u[[j,row]] = u[[row,j]]
    lj = np.eye(N)
    gamma = u[j+1:,j]/ u[j,j]
    lj[j+1:,j] = -gamma
    u = lj @ u

    lj[j+1:,j] = gamma

    l = l @ lj

    pj =np.eye(N)
    pj[[j,row]] = pj[[row,j]]
    p = p @ np.transpose(pj)

  return  l, u, p

In [47]:
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L, u, P = diy_lu_column_pivot(a)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.727 1.    0.    0.    0.   ]
 [1.    0.857 0.41  1.    0.    0.   ]
 [1.    0.941 0.178 0.426 1.    0.   ]
 [1.    0.5   0.706 0.835 0.789 1.   ]] 

u
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  0.000e+00  2.776e-17  2.421e-02  4.866e-02  6.961e-02]
 [ 0.000e+00  2.220e-16 -2.317e-17  0.000e+00 -6.462e-04 -1.516e-03]
 [ 0.000e+00 -1.751e-16  6.463e-18  0.000e+00  0.000e+00  6.730e-06]] 

L@u
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    0.75  0.429 0.3   0.231 0.188]
 [3.    1.364 0.779 0.458 0.253 0.111]
 [3.    1.071 0.652 0.473 0.375 0.313]
 [3.    0.882 0.517 0.366 0.283 0.23 ]
 [3.    1.875 1.467 1.262 1.138 1.055]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.8

In [48]:
a1 = a.copy()
a1[1, 1] = 3

L, u , P = diy_lu_column_pivot(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.727 1.    0.    0.    0.   ]
 [1.    0.857 0.088 1.    0.    0.   ]
 [1.    0.941 0.038 0.208 1.    0.   ]
 [1.    0.    0.151 0.514 0.641 1.   ]] 

u
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.776e-17 -9.247e-02 -1.485e-01 -1.856e-01]
 [ 0.000e+00  2.220e-16 -1.427e-17  0.000e+00  1.841e-03  3.821e-03]
 [ 0.000e+00 -1.424e-16  3.391e-18  0.000e+00  0.000e+00 -1.233e-05]] 

L@u
 [[ 3.     3.     3.     3.     3.     3.   ]
 [ 3.     0.75   0.429  0.3    0.231  0.188]
 [ 3.     1.364 -0.506 -0.892 -1.132 -1.295]
 [ 3.     1.071  0.652  0.424  0.292  0.206]
 [ 3.     0.882  0.517  0.366  0.284  0.232]
 [ 3.     3.     2.752  2.661  2.605  2.567]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 

In [49]:
a2 = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4]])

L, u, P = diy_lu_column_pivot(a2)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")
print("P\n", P, "\n")
print("P@L@u\n", P@L@u, "\n")
print("a\n",a, "\n")

L
 [[ 1.     0.     0.     0.   ]
 [ 0.556  1.     0.     0.   ]
 [ 0.444  0.167  1.     0.   ]
 [ 0.889 -0.333  0.455  1.   ]] 

u
 [[ 9.     9.     3.   ]
 [ 0.    -6.     1.333]
 [ 0.     0.    -1.222]
 [ 0.     0.     0.   ]] 

L@u
 [[ 9.     9.     3.   ]
 [ 5.    -1.     3.   ]
 [ 4.     3.     0.333]
 [ 8.    10.     1.667]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

P
 [[0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]] 

P@L@u
 [[ 8.    10.     1.667]
 [ 4.     3.     0.333]
 [ 9.     9.     3.   ]
 [ 5.    -1.     3.   ]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 



# 3. $LU$ factorization column pivoting and reconstruction
When we premultply $A$ by elementary permutation matricex$P_j$( to find a good pivot) and the premultply by lower triangular elemetary matrices $\Lambda_j$  it is trasfrommed in an  upper triangular matrix $U$

$$  \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  = U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$
$$  A  = (\Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1)^{-1} U$$

The inverse of a product of matrices is the revesre product of inverses 
$$  A  = ( P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\Lambda_{n-1}^{-1} ) U$$

So the $LU$ column pivot factorization is 
$$  A  = L U$$
with
$$ U = \Lambda_{n-1} P_{n-1} \ldots \Lambda_2 P_2 \Lambda_1 P_1 A  $$



$$ L  = P_1^{-1} \Lambda_1^{-1} P_2^{-1} \Lambda_2^{-1} \ldots  P_{n-1}^{-1}\Lambda_{n-1}^{-1} $$

the inverse a of a Lower Triangular Elemetary Matrix is minus the matrix $\Lambda_i^{-1} = - \Lambda_i$ and the inverse of a Permutation Matrix (in particular an elementary permutation matrix)  is its transpose [math.stackexchange](
        https://math.stackexchange.com/questions/98549/the-transpose-of-a-permutation-matrix-is-its-inverse#:~:text=Taking%20the%20transpose%20of%20P,Pt%3DP%E2%88%921.)  $P_i^{-1}=P_i^{t}$, so

$$ L  =  P_1^{t} (-\Lambda_1) P_2^{t} (-\Lambda_2) \ldots  P_{n-1}^{t} (-\Lambda_{n-1}) $$


In [50]:
def diy_lu_column_pivot_reconstruct(a):
  N = a.shape[0]
  u = a.copy()
  L = np.eye(N)
  P = np.eye(N)

  for j in range(1, N-1):
    row = np.argmax(np.abs(u[j:,j]))+j
    u[[j,row]] = u[[row,j]]
    Pj = np.eye(N)
    Pj[[j,row]] = Pj[row,j]
    Lj = np.eye(N)
    gamma = u[j+1:,j]/u[j,j]
    Lj[j+1:,j] = -gamma
    u =Lj @ u
    Lj[j+1:,j] = gamma
    L= L @ np.transpose(Pj) @ Lj
  return  L, u

In [51]:
N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

L, u = diy_lu_column_pivot_reconstruct(a)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a\n",a, "\n")

L
 [[1.    1.    0.    2.    0.    0.   ]
 [0.    1.    0.    1.    0.    0.   ]
 [0.    1.727 0.    1.727 0.    0.   ]
 [0.    1.571 0.    1.571 0.    0.   ]
 [0.    1.471 0.98  2.812 0.    0.   ]
 [0.    1.4   0.92  2.337 0.    0.   ]] 

u
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 3.000e+00  1.875e+00  1.364e+00  1.071e+00  8.824e-01  7.500e-01]
 [ 1.286e+00  0.000e+00 -1.270e-01 -1.435e-01 -1.383e-01 -1.286e-01]
 [-2.888e-01  0.000e+00  0.000e+00 -3.503e-03 -5.357e-03 -6.188e-03]
 [ 3.333e-01  0.000e+00  0.000e+00  0.000e+00 -1.585e-04 -2.976e-04]
 [-3.266e-02  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -5.608e-07]] 

L@u
 [[5.422 4.875 4.364 4.064 3.872 3.738]
 [2.711 1.875 1.364 1.068 0.877 0.744]
 [4.683 3.239 2.355 1.845 1.515 1.285]
 [4.261 2.946 2.143 1.678 1.378 1.169]
 [4.859 2.757 1.881 1.425 1.147 0.96 ]
 [4.708 2.625 1.792 1.36  1.095 0.917]] 

a
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.8

In [52]:
a1 = a.copy()
a1[1, 1] = 3

L, u, = diy_lu_column_pivot_reconstruct(a1)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a1, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - a1\n",L@u - a1, "\n")

L
 [[1.    1.    2.    0.    0.    0.   ]
 [0.    1.    1.    0.    0.    0.   ]
 [0.    1.455 2.455 0.    0.    0.   ]
 [0.    1.357 2.986 0.    0.    0.   ]
 [0.    1.294 2.737 0.972 0.    0.   ]
 [0.    1.25  2.584 0.    0.    0.   ]] 

u
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 3.000e+00  3.000e+00  1.364e+00  1.071e+00  8.824e-01  7.500e-01]
 [ 1.636e+00  0.000e+00  2.625e-01  1.652e-01  1.162e-01  8.766e-02]
 [ 1.704e+00  0.000e+00  0.000e+00 -2.301e-02 -2.861e-02 -2.927e-02]
 [-4.198e-01  0.000e+00  0.000e+00  0.000e+00 -2.107e-04 -3.474e-04]
 [-6.270e-02  0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.116e-06]] 

L@u
 [[ 9.273  6.     4.889  4.402  4.115  3.925]
 [ 4.636  3.     1.626  1.237  0.999  0.838]
 [ 8.38   4.364  2.628  1.964  1.569  1.306]
 [ 8.958  4.071  2.635  1.947  1.544  1.28 ]
 [10.016  3.882  2.483  1.816  1.432  1.182]
 [ 7.978  3.75   2.383  1.766  1.403  1.164]] 

a2
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364

In [53]:
a2 = np.array([[4,3,1], [5,7,0], [9,9,3], [8,2,4]])

L, u, = diy_lu_column_pivot_reconstruct(a2)

print("L\n",L, "\n")
print("u\n",u, "\n")
print("L@u\n", L@u, "\n")
print("a2\n",a2, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print("L@u - a2\n",L@u - a2, "\n")

L
 [[1.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.222 0.    0.   ]] 

u
 [[4.    3.    1.   ]
 [9.    9.    3.   ]
 [6.    0.    3.333]
 [2.2   0.    0.   ]] 

L@u
 [[4.    3.    1.   ]
 [0.    0.    0.   ]
 [0.    0.    0.   ]
 [2.    2.    0.667]] 

a2
 [[4 3 1]
 [5 7 0]
 [9 9 3]
 [8 2 4]] 

L@u - a2
 [[ 0.     0.     0.   ]
 [-5.    -7.     0.   ]
 [-9.    -9.    -3.   ]
 [-6.     0.    -3.333]] 

